In [2]:
import warnings

import opendatasets as od
import pandas as pd
from password_strength import PasswordStats
from tqdm import tqdm

tqdm.pandas()
warnings.filterwarnings("ignore")

In [3]:
database_url = "https://www.kaggle.com/datasets/wjburns/common-password-list-rockyoutxt"
od.download(database_url, data_dir="./raw_dataset")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Your Kaggle Key:Your Kaggle Key:Dataset URL: https://www.kaggle.com/datasets/wjburns/common-password-list-rockyoutxt


In [4]:
df = pd.read_csv(
    "raw_dataset/common-password-list-rockyoutxt/rockyou.txt",
    header=None,
    names=["password"],
    sep="\t",
    encoding="ISO-8859-1",
)
df.head()

,password
0,123456
1,12345
2,123456789
3,password
4,iloveyou


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14344097 entries, 0 to 14344096
Data columns (total 1 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   password  object
dtypes: object(1)
memory usage: 109.4+ MB


In [6]:
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14344094 entries, 0 to 14344096
Data columns (total 1 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   password  object
dtypes: object(1)
memory usage: 218.9+ MB


In [7]:
def cal_strength(text) -> int:
    return PasswordStats(text).strength()

In [8]:
df["strength"] = df["password"].progress_apply(lambda x: cal_strength(x))
df.head()

100%|██████████| 14344094/14344094 [01:22<00:00, 173293.51it/s]


,password,strength
0,123456,0.172331
1,12345,0.128996
2,123456789,0.316992
3,password,0.249543
4,iloveyou,0.249543


In [10]:
df.to_parquet("./data/passwords.gzip", compression="gzip", engine="pyarrow")